# Hyperspectral Imagery Gap Detection, A Flight Assessment Tool
#### ENST30002 Semester 1, 2021
#### Author: Lily Aoi Fujii


### import libraries

In [1]:
from shapely.geometry import LineString, Point, Polygon
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import rasterio
from rasterio.plot import show
import numpy as np
from zipfile import ZipFile
from lxml import html
import math
import datetime as dt
from collections import defaultdict
import tkinter as tk
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib.figure import Figure
from tkinter.filedialog import askopenfilename
from functools import partial
from matplotlib.widgets import RectangleSelector  

### processing gps data

In [2]:
'''
Takes gps file as an input and returns the file in the pandas data frame format.
The gps file must have the following colums: 
"Roll", "Pitch", "Yaw", "Lat", "Lon", "Alt", "Time", "Seconds"
'''
def readGps(gpsfile):
    gps = pd.read_csv(gpsfile, sep='\s+')
    gps = pd.DataFrame(gps).reset_index()
    ## check the format of gps.txt, adjust to fit to more format
    gps.columns = ["Roll", "Pitch", "Yaw", "Lat", "Lon", "Alt", "Time", "Seconds"]
    return gps

'''
Filter the pandas data frame (gps file) by the given longitude nad latitude.
Returns the pandas data frame within the given longitude and latitude.
'''
def filterArea(gps, maxLat, minLat, maxLon, minLon):
    return gps.loc[gps['Lat'].between(minLat, maxLat) & gps['Lon'].between(minLon, maxLon)].reset_index(drop=True)

'''
Convert the given pandas data frame of the gps file into geo-dataframe. 
'''
def toGdf(gps):
    # https://gis.stackexchange.com/questions/174159/converting-pandas-dataframe-to-geodataframe
    geometry = gps[['Lon', 'Lat']].apply(Point, axis=1)
    return GeoDataFrame(gps, crs="EPSG:4326", geometry=geometry)

HIGH_IDX = 1
LOW_IDX = 0

##find more efficient way
##check the empty version of gdf

'''
filter the given data frame by the column value given in the filter dictionary.
'''
def updateGdf(gdf_before, filter_dict):
    
    gdf_after = gdf_before.copy(deep=True)
    
    # no need to filter
    if (not filter_dict):
        return gdf_before
    
    for i in filter_dict.keys():  # filter is 2d array ## consider data structure later
        
        if(i=='Alt'):
            gdf_after = gdf_after[gdf_after[i].between(filter_dict[i][LOW_IDX], filter_dict[i][HIGH_IDX])]
        else:
            gdf_after = gdf_after[np.logical_not(gdf_after[i].between(filter_dict[i][LOW_IDX], filter_dict[i][HIGH_IDX]))]
            
        print("beforeupdate")
        print(gdf_before)
        print("afterupdate")
        print(gdf_after)
        
    return gdf_after


### processing polygon data

In [3]:
# get polygon coords from kmz, polygon has to be single square

## can modify the condition of the "same" coordinates (exact or approximate)
'''Returns true if the given two coordinates 
in string format have the same value'''
def sameCoords(coord1, coord2):
    coord1_lst = coord1.split(",")
    coord2_lst = coord2.split(",")
    # both coodinates should be in the same dimension
    if (len(coord1_lst) != len(coord2_lst)):
        return False;
    for i in range(len(coord1_lst)):
        # coordinates did not match
        if(float(coord1_lst[i]) != float(coord2_lst[i])):
            return False;
    # coordinates matched
    return True;

'''Returns the list of all coordinates in string format (e.g. ["a,b,c d,e,f"]).
if there's no coordinate, it returns an empty list'''
def getCoords(filename):
    # https://dmuhs.blog/2018/09/14/parsing-kmz-track-data-in-python/
    # extract coordinates of the polygon
    kmz = ZipFile(filename, 'r')
    kml = kmz.open('doc.kml', 'r').read()
    doc = html.fromstring(kml)
    # assume there's only one polygon in the file
    return doc.cssselect('coordinates')

## check the format and content of the kml file including polygon shape
''' Find boundaries of the poligon in the given kmz file. 
Add the longitudes and latitudes to the given lists'''
def getPolygonBoundaries(filename, lon, lat):
    coordinates = getCoords(filename)
    # information about coordinates is found
    print(coordinates)
    if len(coordinates):
        # coords_lst contains each coordinate in string
        coords_lst = coordinates[0].text_content().split()
        
        # This is for a single square polygon when 4 coordinates for the polygon are found
        #if (len(coords_lst) == 5 and sameCoords(coords_lst[0], coords_lst[4])):
        
        # may not be square but checking that it's ring (& number of vertex >= 4), only takes max&min values of lon&lat
        if sameCoords(coords_lst[0], coords_lst[-1]) and len(coords_lst) >= 5: 
            # save the coordinates(lat and lon) in float
            # assume coordinates are always in the (lon, lat) format
            for i in coords_lst:
                i_lst = i.split(",")
                lon.add(float(i_lst[0]))
                lat.add(float(i_lst[1]))
        else:
            print("polygon is in acceptable shape. The number of vertex have to be greater than 4.\n")
            exit();
    else:
        print("error: coordinates not found.\n")
        exit();   
        
'''
Returns the longitude and latitude boundaries from from the given polygon file.
out of the four points of the square, ony the highest and lowest logitude and latitude valuee
will be returned.
'''
def upload_poly_coords(filename):
    lon = set()
    lat = set()
    getPolygonBoundaries(filename, lon, lat)
    #lon lower (west, left) #lon higher (east, right) #lat further from the equator (south, down) #lat closer to the equator (north, up)
    return min(lon), max(lon), min(lat), max(lat)


In [4]:

def points_in_poly(filename, gdf):
    lon = []
    lat = []
    getPolygonBoundaries2(filename, lon, lat)
    from geopandas.tools import sjoin
    polygon = Polygon(zip(lon, lat))
    polygon = gpd.GeoDataFrame(index=[0],  crs="EPSG:4326", geometry=[polygon])  
    pointInPolys = sjoin(gdf, polygon, how='left')
    return pointInPolys

# same functionality as the getPolygonBoundaries function but this is applicable to "list" not "set"
def getPolygonBoundaries2(filename, lon, lat):
    coordinates = getCoords(filename)
    # information about coordinates is found
    print(coordinates)
    if len(coordinates):
        # coords_lst contains each coordinate in string
        coords_lst = coordinates[0].text_content().split()
        
        # may not be square but checking that it's ring, only takes max&min values of lon&lat
        if sameCoords(coords_lst[0], coords_lst[-1]) and len(coords_lst) >= 5: 
            # save the coordinates(lat and lon) in float
            # assume coordinates are always in the (lon, lat) format
            for i in coords_lst:
                i_lst = i.split(",")
                lon.append(float(i_lst[0]))
                lat.append(float(i_lst[1]))
        else:
            print("polygon is in acceptable shape. The number of vertex have to be greater than 4.\n")
            exit();
    else:
        print("error: coordinates not found.\n")
        exit();   


### grouping trajectory

In [5]:
# use time gap to separate one gdf into groups (each group should represent one cotinuous line on the map) 
def grouped_trajectory(gdf, group_i_lst, grouped_lst):
    # base, no need for split
    if len(group_i_lst) == 0:
        if not gdf.empty:
            grouped_lst.append(gdf)
    else:
        center_i = group_i_lst[len(group_i_lst) // 2] 
        # just center point is left
        if len(group_i_lst) == 1:
            group_i_lst1 = []
            group_i_lst2 = []
        else:
            group_i_lst1 = group_i_lst[:len(group_i_lst) // 2]
            # center point and one another index are left
            if (len(group_i_lst) == 2):
                group_i_lst2 = []
            # center point and 2+ index are left
            else:
                group_i_lst2 = list(map(lambda x: x - center_i, group_i_lst[len(group_i_lst) // 2 + 1:]))

        # split df into half at the center
        gdf_half1 = gdf.iloc[:center_i, :]
        gdf_half2 = gdf.iloc[center_i:, :]

        # recursively group the instances 
        grouped_trajectory(gdf_half1, group_i_lst1, grouped_lst)
        grouped_trajectory(gdf_half2, group_i_lst2, grouped_lst)



### swath calculation

In [6]:
# calculates half of the swath from FOV and altitude
def fov_to_swath(FOV, altitude):
    swath = (2 * altitude) * math.tan(math.radians(FOV) / 2)
    return swath / 2

# calculate change in meter per longitude at the given latitude
def get_m_per_lon(lat):
    earth_radius = 6371000
    r = earth_radius * math.cos(math.radians(lat)) ####check this radian notation
    return 2 * r * math.pi / 360 # meter per lon degree
    
# convert the meter to the amount of lat/lon to shift from the given point. (half of the swath)
def m_to_lon(meter, lat):
    t = get_m_per_lon(lat)
    return meter / t
    
# calculate the overlap of the swath from the half of the swath in meter and two coordinates of the data point (not the lon of the edge of swath)
def get_overlap(h_swath, lon1, lon2, lat):
    if lon1 > lon2:
        tmp =lon1
        lon1 = lon2
        lon2 = tmp
     
    #print("lon diff from get overlap")
    #print(lon_diff, lon1 + m_to_lon(h_swath, lon1), lon2 -  m_to_lon(h_swath, lon2))
    h_swath_lon = m_to_lon(h_swath, lat)
    coverage = LineString([(lon1 + h_swath_lon, lat), (lon2 - h_swath_lon, lat)])
    # differnce of lon of the two closest swath edges in longitude
    lon_diff = (lon1 + h_swath_lon) - (lon2 -  h_swath_lon)
    
    # have overlap
    if lon_diff > 0:
        t = get_m_per_lon(lat)
        m_diff = t * lon_diff
        return m_diff / (2 * h_swath), coverage
    # no overlap
    else:
        return 0, coverage

# stores the swath line of the given latitude, longitude and the half of the swath width.
def store_swath_edge(curr_avg_lat, lon, h_swath, line_lst):
    line =  LineString([(lon + m_to_lon(h_swath, lon), curr_avg_lat), (lon - m_to_lon(h_swath, lon), curr_avg_lat)])
    line_lst.append(line)


### sampling data frame to avoid overplotting and reduce computation time

In [7]:
class sampling:
    def __init__(self, gdf, fov):
        self.prev_time = gdf['Time'][0]
        self.prev_milisec = gdf['Seconds'][0]
        self.interval = 0.5
        self.lat_interval = 2
        self.prev_lat = gdf['Lat'][0]
        self.middle_counter = 0
        self.FOV = fov 
        
    def time_diff(self, previous, current, previous_milisec, current_milisec):
        current_dt = dt.datetime.strptime(current, '%H:%M:%S')
        previous_dt = dt.datetime.strptime(previous, '%H:%M:%S')
        
        if (current_dt < previous_dt):
            print("time frame in reverse order")
            exit()
        else:
            diff = (current_dt - previous_dt) 
            return (diff.seconds * 1000000 + (int(current_milisec) - int(previous_milisec))) / 1000000
    
    #interval [x...x].[xxxxxx]
    #time  hh:mm:ss, milisec mmmmmm
    def track_time(self, curr_time, prev_time, curr_milisec, prev_milisec):
        
        diff_sec = self.time_diff(prev_time, curr_time, prev_milisec, curr_milisec)
        if (diff_sec >= self.interval):
            # select the current point to display on the map
            # curr_time will be the new prev_time
            return True, curr_time, curr_milisec, self.group_gdf(diff_sec)
        
        else:
            return False, prev_time, prev_milisec, self.group_gdf(diff_sec) 

    def track_lat(self, curr_time, prev_time, curr_milisec, prev_milisec, curr_lat, prev_lat):
        diff_sec = self.time_diff(prev_time, curr_time, prev_milisec, curr_milisec)
        diff_lat = abs(curr_lat * 10000 - prev_lat * 10000)        #interval is (0.0001 * x) so make this into x  
        
        if (diff_lat >= self.lat_interval):
            # select the current point to display on the map
            # curr_time will be the new prev_time, curr_lat will ne the new prev_lat
            return True, curr_time, curr_milisec, self.group_gdf(diff_sec), curr_lat
        
        else:
            return False, prev_time, prev_milisec, self.group_gdf(diff_sec), prev_lat
    
    def group_gdf(self, diff_sec):
        # found the boundary between differnt trajectory
        # (took double of the interval time -> more likely to moved to differnt line)
        return diff_sec > self.interval * 2
    
    def sample_data(self, gdf):
        
        # list of Boolean value to drop the rows taht were not sampled
        plot_lst = []
        # index where the new trajectory line started
        group_lst = []
        # count the number of sampled points
        counter = 0
        # count the number of trajectory lines
        group_counter = 1
        
        lat_lon_dict = defaultdict(list)
        # the range of the latitude in the data frame
        lat_total_range = abs(max(gdf['Lat']) * 10000 - min(gdf['Lat']) * 10000)
        # the maximum number of latitude interval
        block_upper_bound = lat_total_range // self.lat_interval + 1
    
        # avg lat for each instance
        avg_lat = [] # store in the df
        # list of avg latitude
        avg_lat_lst = []
        
        # store swath
        line_lst = []
        #store coverage
        overlap = []
        sampled_to_swath_dict = {}
        overlapped_line = []
        
        diff_line_cover = 0
                
        # lines between two trajectory lines
        lat_middle = []
        lon_middle = []
        sampled_to_middle_dict = defaultdict(list)
    
        for i in range(len(gdf)):
            
            plot, self.prev_time, self.prev_milisec, group, self.prev_lat = self.track_lat(gdf['Time'][i], self.prev_time, gdf['Seconds'][i], self.prev_milisec, gdf['Lat'][i], self.prev_lat)
            plot_lst.append(plot)
            
            # index of the first row of a new group
            if group:
                # index of the starting point of a new group
                group_lst.append(counter) # index is one smaller
                group_counter += 1
                
            if plot: 
                counter += 1
                if group_counter == 1:
                    diff_line_cover += 1
                
                # check previous lon with similar lat, calculate avg (middle point) and store
                self.get_same_lat_points(gdf['Alt'][i], gdf['Lat'][i], gdf['Lon'][i], lat_middle, lon_middle, min(gdf['Lat']), block_upper_bound,
                                         group_counter, lat_lon_dict, counter, sampled_to_middle_dict, avg_lat, avg_lat_lst, line_lst,
                                         overlap, sampled_to_swath_dict, overlapped_line)
        
        # convert this to gdf for plotting
        middle_trajectory_df = pd.DataFrame({'Lat': lat_middle, 'Lon': lon_middle})#, 'Points_around1': points_around1, 'Points_around1': points_around1})
        middle_trajectory_gdf = toGdf(middle_trajectory_df).reset_index(drop=True)

        line_gdf = gpd.GeoDataFrame(crs="EPSG:4326", geometry=line_lst)
        
        coverage_gdf = gpd.GeoDataFrame(crs="EPSG:4326", geometry=overlapped_line)
        coverage_gdf['coverage'] = pd.Series(overlap)
        
        print("overlap list")
        print(overlap)
        
        return plot_lst, group_lst, middle_trajectory_gdf, sampled_to_middle_dict, line_gdf, sampled_to_swath_dict, coverage_gdf, diff_line_cover

    
    def get_same_lat_points(self, alt, row_lat, row_lon, lat_middle, lon_middle, min_lat, block_upper_bound, group_counter, lat_lon_dict,
                            counter, sampled_to_middle_dict, avg_lat, avg_lat_lst, line_lst, overlap, sampled_to_swath_dict, overlapped_line):
        # get dict search key from each lat value
        block = (row_lat * 10000 - min_lat * 10000) // self.lat_interval #0-
        lat_key = min_lat * 10000 + self.lat_interval * (block + 1 / 2) # key value is the actual lat value x 10000
        
        # store average lat for each instance and for indexing (no duplicate) for filtering ([lat1, lat1, ... lat2, ...] and [lat1, lat2, lat3, ...])
        avg_lat.append(lat_key * pow(10, -4)) #add avg lat to the list for df # convert is back to normal lat value
        if lat_key not in avg_lat_lst:
            avg_lat_lst.append(lat_key * pow(10, -4)) # convert is back to normal lat value
        
        prev_lon = lat_lon_dict[lat_key] #list of [groupnum, lon, index_for_df]
        
        '''
        def find_prev_lon(group_counter, prev_lon):
            # find previous lon value with the same lat key
            # find lon from the previous group with the same lat 
            return len(prev_lon) > 0 and prev_lon[-1][0] == group_counter - 1 #previous group number of the list in prev_lon 
        '''
        
        def find_prev_lon(prev_lon, row_lon):
            # find previous lon value with the same lat key
            # find lon from the previous group with the same lat 
            if len(prev_lon) > 0: #have at least one previous longitude point with the same avg latirude 
                # sort the elements of prev_lon based on the longitude value (index 1 as element is [group, lon, index in df])
                sorted_lon = sorted(prev_lon, key = lambda x: x[1])
                # find the closest lon from the closest group recorded
                closest = sorted_lon.pop()
                while sorted_lon:
                    current = sorted_lon.pop()
                    # the difference of the longitude is greater
                    if abs(current[1] - row_lon) >= abs(closest[1] - row_lon):
                        return closest
                    closest = current
                return closest
            
            else:
                return []
            
        
        def store_middle_points(group_counter, closest_lon, row_lon, lon_middle, lat_middle, lat_key, lat_lon_dict, counter, sampled_to_middle_dict):
            # take avg, store point
            self.middle_counter += 1
            
            avg_lon = (closest_lon[1] * pow(10, 10) + row_lon * pow(10, 10)) / 2 #actual lon * 10^10 
            lon_middle.append(avg_lon * pow(10, -10)) # convert it back to normal lon value
            lat_middle.append(lat_key * pow(10, -4)) # convert is back to normal lat value
            
            # add index of point from middle df that corresponds to the index of the point around it
            # store the middle index correspond to the sampled index
            # current point with same lat (the latter point added out of the two points creating the middle trajectpry point)
            sampled_to_middle_dict[counter - 1].append(self.middle_counter - 1)
    
            # previous point with same lat (point on the left side of the middle point)
            #trajectory point from previous group (longitude of the same latitude point) in the sampled data to middle 
            #sampled_to_middle_dict[lat_lon_dict[lat_key][-1][2]].append(self.middle_counter - 1) # sample s,t -> middle i
            sampled_to_middle_dict[closest_lon[2]].append(self.middle_counter - 1) # sample s,t -> middle i
        
        h_swath = fov_to_swath(self.FOV, alt)
        
        closest_lon = find_prev_lon(prev_lon, row_lon)
        if closest_lon:
            # found previous lon
            store_middle_points(group_counter, closest_lon, row_lon, lon_middle, lat_middle, lat_key, lat_lon_dict, counter, sampled_to_middle_dict)

            ## use dict for now, can be implemented in different structure e.g. hash?
            # calculate swath and coverage at the same time and save line to df for plotting

            # prevlon, current lon -> edge -> (swath), coverage
            #overlap.append(get_overlap(self.h_swath, prev_lon[-1][1], row_lon, lat_key * pow(10, -4)))
            cover_diff = get_overlap(h_swath, closest_lon[1], row_lon, lat_key * pow(10, -4))
            
            overlap.append(cover_diff[0]) # coverage percentage
            overlapped_line.append(cover_diff[1]) # line of the overlapped area between two swath
            
        # avglat, edge ->linelst -> line df
        store_swath_edge(lat_key * pow(10, -4), row_lon, h_swath, line_lst)
        # index of sampled df -> line_df
        sampled_to_swath_dict[counter - 1] = len(line_lst) - 1
        
        # add to the lat lon dict value of the trajectory point
        lat_lon_dict[lat_key].append([group_counter, row_lon, counter-1])
      
    
    def sample_gdf(self, gdf):
        # iterate through the gdf, get the index of the row that should be plot, then iloc the gdf using a list of index
        gdf['sampled'], group_i_lst, middle_trajectory_gdf, sampled_to_middle_dict, line_gdf, sampled_to_swath_dict, coverage_gdf, diff_line_cover = self.sample_data(gdf)
        
        sampled = gdf.drop(gdf[gdf.sampled != True].index).reset_index()
        
        print("sampled:")
        print(sampled)
        return sampled, group_i_lst, middle_trajectory_gdf, sampled_to_middle_dict, line_gdf, sampled_to_swath_dict, coverage_gdf, diff_line_cover
        

### visualisation (GUI)

In [ ]:
#https://pythonprogramming.altervista.org/create-more-windows-with-tkinter/?doing_wp_cron=1616544110.7440760135650634765625
#embed in tk: https://matplotlib.org/3.1.0/gallery/user_interfaces/embedding_in_tk_sgskip.html

class window_upload:
    
    def addLabel(self, txt):
        label = tk.Label(text=txt)
        label.pack()
        return label;
    
    def upload(self, window):
        self.imgfilepath = str()
        self.gpsfilepath = str()
        self.polyfilepath = str()
        
        self.fov = float() # default
        
        frame1 = tk.Frame(window)
        frame1.pack(side = tk.RIGHT, fill = tk.BOTH, expand = tk.Y, padx=20, pady=20)
        
        frame2 = tk.Frame(window)
        frame2.pack(side = tk.LEFT, fill = tk.Y, padx=20, pady=20)
        tk.Label(frame2, text = "Select Files to Upload", font=('calibre',10, 'bold')).pack(side=tk.TOP)
        
        frame_fov = tk.Frame(frame2)
        frame_fov.pack(side = tk.TOP, fill = tk.Y, padx=20, pady=20)
        frame_img = tk.Frame(frame2)
        frame_img.pack(side = tk.TOP, fill = tk.Y, padx=20, pady=20)
        frame_gps = tk.Frame(frame2)
        frame_gps.pack(side = tk.TOP, fill = tk.Y, padx=20, pady=20)
        frame_poly = tk.Frame(frame2)
        frame_poly.pack(side = tk.TOP, fill = tk.Y, padx=20, pady=20)
        
        #https://www.geeksforgeeks.org/python-gui-tkinter/
        scrollbar = tk.Scrollbar(frame1)
        scrollbar.pack(side = tk.RIGHT, fill = tk.Y)
        scrollbarx = tk.Scrollbar(frame1, orient = tk.HORIZONTAL)
        scrollbarx.pack(side = tk.BOTTOM, fill = tk.X)
        
        label = tk.Label(frame1, text = "Upload Status", font=('calibre',10, 'bold'))
        label.pack(side=tk.TOP)
        
        status_labels = tk.Listbox(frame1, yscrollcommand = scrollbar.set, xscrollcommand = scrollbarx.set)
        status_labels.insert(tk.END, "File upload status is shown here.")
        status_labels.pack(side = tk.BOTTOM, fill = tk.BOTH, expand = tk.Y)
        scrollbar.config(command = status_labels.yview)
        scrollbarx.config(command = status_labels.xview)
        
        def upload_img():
            self.imgfilepath = askopenfilename()
            print(self.imgfilepath)
            if (self.imgfilepath):
                print("got file")
                status_labels.insert(tk.END, f"{self.imgfilepath} is added as a background image.")
                proceed()
                 
        def upload_gps():
            self.gpsfilepath = askopenfilename()
            print(self.gpsfilepath)
            if (self.gpsfilepath):
                print("got file")
                status_labels.insert(tk.END, f"{self.gpsfilepath} is added as a GPS file.")
                proceed()
                
        def upload_polygon():
            self.polyfilepath = askopenfilename()
            print(self.polyfilepath)
            if (self.gpsfilepath):
                print("got file")
                status_labels.insert(tk.END, f"{self.polyfilepath} is added as a polygon file.")
                proceed()    
             
        def add_fov(var):
            self.fov = float(var)
            print(self.fov)
            if (self.fov):
                print("got fov")
                status_labels.insert(tk.END, f"{self.fov} is added as a field of view parameter.")
                proceed()    
            
        def proceed():
            if(self.imgfilepath and self.gpsfilepath and self.polyfilepath and self.fov):
                
                status_labels.insert(tk.END, "Uploading files...")
                status_labels.insert(tk.END, "Reading an image file...")
                window.update_idletasks()
                
                raster = rasterio.open(self.imgfilepath)
                status_labels.insert(tk.END, "...Completed.")
                status_labels.insert(tk.END, "Reading a polygon file...")
                window.update_idletasks()
                
                minLon, maxLon, minLat, maxLat = upload_poly_coords(self.polyfilepath)
                
                status_labels.insert(tk.END, "...Completed.")
                status_labels.insert(tk.END, "Reading a GPS file...")
                window.update_idletasks()
                
                gps = readGps(self.gpsfilepath)
                status_labels.insert(tk.END, "...Completed.")
                status_labels.insert(tk.END, "Obtaining GPS points within a polygon...")
                window.update_idletasks()
                
                #filter df before creating geodf
                gps = filterArea(gps, maxLat, minLat, maxLon, minLon)
                gdf = toGdf(gps)
                
                print("before poly")
                print(gdf)
                gdf = points_in_poly(self.polyfilepath, gdf)
                print("after poly")
                print(gdf)
                # drop the row outside of the polygon
                gdf = gdf.loc[gdf["index_right"].isnull() == False].reset_index()
                print("after musking")
                print(gdf)
                print("Obtained points within polygon.")
        
                status_labels.insert(tk.END, "...Completed.")
                status_labels.insert(tk.END, "Completed file upload.")
                window.update_idletasks()
                
                self.window.button = tk.Button(frame2, text="proceed", command=lambda: self.new_window(window_image, raster, gdf, self.fov))
                self.window.button.pack(side = tk.BOTTOM)
        
        tk.Label(frame_fov, text = "Field of View: ", font=('calibre',10, 'bold')).pack(side=tk.LEFT)
        button = tk.Button(frame_fov, text="Enter", command=lambda: add_fov(var.get())).pack(side=tk.RIGHT)         
        var = tk.StringVar()
        tk.Entry(frame_fov, width = 4, textvariable=var).pack(side = tk.RIGHT, padx=10)
        
        tk.Label(frame_img, text = "Image file:    ", font=('calibre',10, 'bold')).pack(side=tk.LEFT)
        self.window.imgbutton = tk.Button(frame_img, text="Select", command=upload_img).pack(side=tk.RIGHT)         
        
        tk.Label(frame_gps, text = "GPS file:      ", font=('calibre',10, 'bold')).pack(side=tk.LEFT)
        self.window.gpsbutton = tk.Button(frame_gps, text="Select", command=upload_gps).pack(side=tk.RIGHT)         
        
        tk.Label(frame_poly, text = "Polygon file:  ", font=('calibre',10, 'bold')).pack(side=tk.LEFT)
        self.window.imgbutton = tk.Button(frame_poly, text="Select", command=upload_polygon).pack(side=tk.RIGHT)         
    
    def __init__(self, window):
        self.window = window
        self.window.geometry('800x500')
        self.window.title("Hyperspectral Remote Sensing & Precision Agriculture Laboratory - Flight Assessment Tool - File Upload")
        self.upload(window)
      
    def new_window(self, _class, raster, gdf, fov):
        self.new = tk.Toplevel(self.window)
        _class(self.new, raster, gdf, fov)
    
    
class window_image:
    
    #min/max [entry]
    def make_entry(self, frame, limit):
        row = tk.Frame(frame)
        label = tk.Label(row, text = limit, font=('calibre',10, 'bold'))
        var = tk.StringVar()
        entry = tk.Entry(row, width = 4, textvariable=var)
        label.pack(side=tk.LEFT)
        entry.pack(side=tk.LEFT, padx=5)
        row.pack(side=tk.TOP)
        
        return var
        ##check error value(not convertible to float)
    
    def add_filter(self, filter_dict, var_name, low, high, gdf):
        
        ## show the current filtering value
        
        if low=='' and high=='':
            if(var_name in filter_dict.keys()):
                filter_dict.pop(var_name)
        else:
            if low=='':
                print('reset entry')
                low = min(gdf[var_name])
            elif high=='':
                print('reset entry')
                high = max(gdf[var_name])
            filter_dict[var_name] = [float(low), float(high)]
            
    #label min[entry] max[entry]
    def make_entryvar(self, filter_lst, frame, var_name, gdf):
        section = tk.Frame(frame)
        section.pack(side = tk.TOP, fill = tk.BOTH, expand = tk.Y, padx = 5, pady = 5)
        head = tk.Frame(section)
        head.pack(side = tk.TOP, fill = tk.BOTH, expand = tk.Y)
        label = tk.Label(head, text = var_name, font=('calibre',10, 'bold'))
        label.pack(side = tk.LEFT)
        low = self.make_entry(section, "min")
        high = self.make_entry(section, "max")
        button = tk.Button(head, text="Apply", command=lambda: self.add_filter(filter_lst, var_name, low.get(), high.get(), gdf)).pack(side = tk.LEFT, padx = 10)         
    
    def __init__(self, window, raster, gdf, fov):
        self.window = window
        self.window.geometry('800x500')
        self.window.title("Hyperspectral Remote Sensing & Precision Agriculture Laboratory - Flight Assessment Tool - Trajectory Image")
        
        #### think another way instead of using 3 copies?
        # sampled gdf that won't change
        s = sampling(gdf, fov)
        self.sampled_gdf, self.group_i_lst, self.middle_trajectory_gdf, self.sampled_to_middle_dict, self.line_gdf, self.sampled_to_swath_dict, self.coverage_gdf, self.diff_line_cover = s.sample_gdf(gdf)

        self.diff_line_cover = len(self.line_gdf) - len(self.coverage_gdf)
        print(f"diff_len:{self.diff_line_cover}, line: {self.line_gdf}, coverage: {self.coverage_gdf}")
        
        print("original gdf:")
        print(gdf)
        
        # list of grouped gdf (each gdf is one trajectory line) 
        self.grouped_lst = []
        grouped_trajectory(self.sampled_gdf, self.group_i_lst, self.grouped_lst)
        print("number of trajectory line")
        print(len(self.grouped_lst))
        #print(self.grouped_lst)
        
        # modified sampled_gdf (modified every time you plot)
        self.gdf = gdf
        
        # roi of self.gdf
        self.roi_gdf = gdf
        
        # filtering entries
        filter_dict = dict()
        frame = tk.Frame(window)
        ###think about another way to use gdf instead of passing on to all the functions############33
        self.make_entryvar(filter_dict, frame, 'Roll', gdf)
        self.make_entryvar(filter_dict, frame, 'Pitch', gdf)
        self.make_entryvar(filter_dict, frame, 'Yaw', gdf)
        self.make_entryvar(filter_dict, frame, 'Alt', gdf)
        
        def update_threshold(val):
            if val:
                self.thres = float(val)
            else:
                self.thres = 0.45
        
        self.thres = 0.45
        section = tk.Frame(frame)
        section.pack(side = tk.TOP, fill = tk.BOTH, expand = tk.Y, padx = 8, pady = 8)
        label = tk.Label(section, text = "coverage threshold (default: 0.45)", font=('calibre',10, 'bold')).pack(side = tk.TOP)
        
        row = tk.Frame(section)
        thres = tk.StringVar()
        entry_thres = tk.Entry(row, width = 4, textvariable=thres)
        entry_thres.pack(side=tk.LEFT, padx=5)
        row.pack(side=tk.TOP)
        button = tk.Button(row, text="Apply", command=lambda: update_threshold(thres.get())).pack(side = tk.RIGHT, padx = 8)         
        
        #calculation
        #self.make_entryvar(filter_dict, frame, 'Speed') ## require further implementaitons for the calculations of speed
        
        fig, ax = plt.subplots(figsize=(15, 15))
        plt.close()
        self.canvas = FigureCanvasTkAgg(fig, master = window)
        
        self.add = True
        #refer to: https://matplotlib.org/stable/gallery/widgets/rectangle_selector.html
        def line_select_callback(eclick, erelease):
            """
            Callback for line selection.
            *eclick* and *erelease* are the press and release events.
            """
            # x: lon, y: lat
            x1, y1 = eclick.xdata, eclick.ydata
            x2, y2 = erelease.xdata, erelease.ydata

            if(self.add == True):
                print("filterROI")
                #filter gdf and get the gps points within roi
                #everytime new roi
                roi_dict = {}
                # order in the form small, large
                #somehow combine with add_filter func????
                def add_and_order_coord(roi_dict, var1, var2, var_name):
                    if (var1 < var2):
                        roi_dict[var_name] = [var1, var2]
                    else:     
                        roi_dict[var_name] = [var2, var1]
                    return roi_dict
                # consider teh case when the selected ROI is greater/smaller than max/min lat, lon -> use max/min value instead
                def get_coordinate_within_polygon(roi_dict, var_name, gdf):
                    if (roi_dict[var_name][0] < min(gdf[var_name])): ##causion: float comparison
                        roi_dict[var_name][0] = min(gdf[var_name])
                    if (roi_dict[var_name][1] > max(gdf[var_name])):
                        roi_dict[var_name][1] = max(gdf[var_name])
                    return roi_dict
                
                roi_dict = add_and_order_coord(roi_dict, x1, x2, 'Lon')
                roi_dict = add_and_order_coord(roi_dict, y1, y2, 'Lat')
                roi_dict = get_coordinate_within_polygon(roi_dict, 'Lon', gdf)
                roi_dict = get_coordinate_within_polygon(roi_dict, 'Lat', gdf)
                self.roi_gdf = updateGdf(self.gdf, roi_dict)
                
                print("selected Roi")
                print(self.roi_gdf)
                print("roi dict")
                print(roi_dict)

            print(f"({x1:.9f}, {y1:.9f}) --> ({x2:.9f}, {y2:.9f})")
        
        #refer to: https://matplotlib.org/stable/gallery/widgets/rectangle_selector.html
        # modified by Aoi Fujii
        # square coordinates added for roi selection. removed if d is pressed before roi selection
        # press 's' to activate the roi selection. press 'q' to deactivate.
        def toggle_selector(event):
            print('Key pressed.')
            
            self.add = True
            print("add mode")
            
            #if event.key == 'd':
            #    print("delete mode")
            #    self.add = False
            
            if event.key == 's':
                print(' RectangleSelector activated.')
                toggle_selector.RS.set_active(True)

            elif event.key == 'q':
                #if toggle_selector.RS.active:
                print(' RectangleSelector deactivated.')
                toggle_selector.RS.set_active(False)
                  
        #refer to: https://matplotlib.org/stable/gallery/widgets/rectangle_selector.html
        # drawtype is 'box' or 'line' or 'none'
        toggle_selector.RS = RectangleSelector(ax, line_select_callback,
                                               drawtype='box', useblit=True,
                                               button=[1, 3],  # disable middle button
                                               minspanx=5, minspany=5,
                                               spancoords='pixels',
                                               interactive=True)

        self.window.button = tk.Button(frame, text="See ROI", command=lambda: self.roi_selection()).pack(side = tk.BOTTOM)
        self.window.button = tk.Button(frame, text="Select ROI", command=lambda: fig.canvas.mpl_connect('key_press_event', toggle_selector)).pack(side = tk.BOTTOM)
        self.window.button = tk.Button(frame, text="Display coverage", command=lambda: self.plot(filter_dict, window, raster, fig, ax, self.canvas, True)).pack(side = tk.BOTTOM) 
        self.window.button = tk.Button(frame, text="Display trajectories", command=lambda: self.plot(filter_dict, window, raster, fig, ax, self.canvas, False)).pack(side = tk.BOTTOM)
        frame.pack(side = tk.LEFT) 
        
    def roi_selection(self):
        # get image correspond to the selected gps
        print("do something")
        print(self.roi_gdf)
        
    def plot(self, filter_dict, window, raster, fig, ax, canvas, plot_middle):
        
        print("filter dict: ")
        print(filter_dict)
        print("before plotting")
        print(self.sampled_gdf)
        #print(self.gdf)
        
        # self.gdf in the window_image class is updated
        self.gdf = updateGdf(self.sampled_gdf, filter_dict)
        print("after update")
        print(self.gdf)
        chosen_index = self.gdf.index.tolist()
        
        ax.clear()
        
        #show raster file
        rasterio.plot.show(raster, ax=ax)
        
        # show trajectory
        self.gdf.plot(ax=ax, color='orangered', markersize=4)
        plt.close()
       
        if plot_middle:
            
            print("chosen index")
            print(len(chosen_index))
            #print(chosen_index)
            
            # selecting index from middle_df that corresponds to the chosen index in sampled_df
            chosen_middle_index = set()
            chosen_swath_index = []
            for i in chosen_index:
                #chosen_middle_index |= {*self.sampled_to_middle_dict[i]}
                chosen_middle_index = chosen_middle_index.union(set(self.sampled_to_middle_dict[i]))
                chosen_swath_index.append(self.sampled_to_swath_dict[i])
            #print("chosen middle index")
            #print(chosen_middle_index)
            
            middle_trajectory = self.middle_trajectory_gdf.iloc[list(chosen_middle_index), :]
            swath_gdf = self.line_gdf.iloc[chosen_swath_index, :]
            
            print("middle trajectory")
            print(middle_trajectory)
            middle_trajectory.plot(ax=ax, markersize=4, color = 'yellow')
            plt.close()
            
            print("swath")
            print(swath_gdf)
            swath_gdf.plot(ax=ax, markersize=0.5, color = 'blue')
            plt.close()
            
            # consider the difference of the index of line_gdf and coverage_gdf so they correspond to the same point
            cover_gdf = self.coverage_gdf.iloc[[x - self.diff_line_cover for x in chosen_swath_index], :]
            
            print("overlap")
            print(cover_gdf)
            # plot the overlapped swath with green if its coverage is over threshold
            cover_gdf.loc[cover_gdf["coverage"] >= self.thres].plot(ax=ax, markersize=0.5, color = 'green')
            plt.close()
            # otherwise plot with red colour
            cover_gdf.loc[cover_gdf["coverage"] < self.thres].plot(ax=ax, markersize=0.5, color = 'red')
            #cover_gdf.plot(ax=ax, markersize=0.5, kind='bar', color=[np.where(cover_gdf["coverage"]>10, 'green', 'red')]) #color = 'green')# 
            plt.close()
        
        ax.set_title ("Trajectory", fontsize=16) 
        canvas.draw()
        canvas.flush_events() ##
        
        toolbar = NavigationToolbar2Tk(canvas, window)
        toolbar.update()
        canvas.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=1)
        canvas._tkcanvas.pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        
window = tk.Tk()
showImage = window_upload(window)
window.mainloop()